# 📝 Exercise M4.02

In the previous notebook, we showed that we can add new features based on the
original feature `x` to make the model more expressive, for instance `x ** 2` or
`x ** 3`. In that case we only used a single feature in `data`.

The aim of this notebook is to train a linear regression algorithm on a
dataset with more than a single feature. In such a "multi-dimensional" feature
space we can derive new features of the form `x1 * x2`, `x2 * x3`, etc.
Products of features are usually called "non-linear" or "multiplicative"
interactions between features.

Feature engineering can be an important step of a model pipeline as long as
the new features are expected to be predictive. For instance, think of a
classification model to decide if a patient has risk of developing a heart
disease. This would depend on the patient's Body Mass Index which is defined
as `weight / height ** 2`.

We load the dataset penguins dataset. We first use a set of 3 numerical
features to predict the target, i.e. the body mass of the penguin.

<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Note</p>
<p class="last">If you want a deeper overview regarding this dataset, you can refer to the
Appendix - Datasets description section at the end of this MOOC.</p>
</div>

In [3]:
import pandas as pd

penguins = pd.read_csv("../datasets/penguins.csv")

columns = ["Flipper Length (mm)", "Culmen Length (mm)", "Culmen Depth (mm)"]
target_name = "Body Mass (g)"

# Remove lines with missing values for the columns of interest
penguins_non_missing = penguins[columns + [target_name]].dropna()

data = penguins_non_missing[columns]
target = penguins_non_missing[target_name]
data.head()

,Flipper Length (mm),Culmen Length (mm),Culmen Depth (mm)
0,181.0,39.1,18.7
1,186.0,39.5,17.4
2,195.0,40.3,18.0
4,193.0,36.7,19.3
5,190.0,39.3,20.6


Now it is your turn to train a linear regression model on this dataset. First,
create a linear regression model.

In [1]:
# Write your code here.
from sklearn.linear_model import LinearRegression

baseline_model = LinearRegression()

Execute a cross-validation with 10 folds and use the mean absolute error (MAE)
as metric.

The `cross_validate` and `cross_val_score` functions have `neg_mean_absolute_error` and no `mean_absolute_error`.

This makes more sense when we think of accuracy. We usually search to **maximize** the **accuracy**. However, we usually want to **minimize** the **MAE**. If we flip the sign of the MAE, we go back again to a maximization problem. And thus, the negative sign of some of the scoring strategies is justified in order to bring all problems to a maximization problem. This is specifically more relevant for hyperparameter optimization tasks.

In [10]:
# Write your code here.
from sklearn.model_selection import cross_val_score

scores = cross_val_score(baseline_model, data, target, cv=10, scoring="neg_mean_absolute_error")

Compute the mean and std of the MAE in grams (g). Remember you have to revert
the sign introduced when metrics start with `neg_`, such as in
`"neg_mean_absolute_error"`.

In [11]:
# Write your code here.
scores = -scores

print(
    "The mean cross-validation MAE is: "
    f"{scores.mean():.3f} ± {scores.std():.3f} "
)

The mean cross-validation MAE is: 337.071 ± 84.868 


Now create a pipeline using `make_pipeline` consisting of a
`PolynomialFeatures` and a linear regression. Set `degree=2` and
`interaction_only=True` to the feature engineering step. Remember not to
include a "bias" feature (that is a constant-valued feature) to avoid
introducing a redundancy with the intercept of the subsequent linear
regression model.

You may want to use the `.set_output(transform="pandas")` method of the
pipeline to answer the next question.

In [17]:
# Write your code here.
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures

polynomial_regression = make_pipeline(
    PolynomialFeatures(degree=3, include_bias=False, interaction_only=True),
    LinearRegression(),
)
polynomial_regression.set_output(transform="pandas")

Pipeline(steps=[('polynomialfeatures',
                 PolynomialFeatures(degree=3, include_bias=False,
                                    interaction_only=True)),
                ('linearregression', LinearRegression())])

Transform the first 5 rows of the dataset and look at the column names. How
many features are generated at the output of the `PolynomialFeatures` step in
the previous pipeline?

In [27]:
# Write your code here.
polynomial_regression[0].fit_transform(data.iloc[:5,:])

,Flipper Length (mm),Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm) Culmen Length (mm),Flipper Length (mm) Culmen Depth (mm),Culmen Length (mm) Culmen Depth (mm),Flipper Length (mm) Culmen Length (mm) Culmen Depth (mm)
0,181.0,39.1,18.7,7077.1,3384.7,731.17,132341.77
1,186.0,39.5,17.4,7347.0,3236.4,687.30,127837.80
2,195.0,40.3,18.0,7858.5,3510.0,725.40,141453.00
4,193.0,36.7,19.3,7083.1,3724.9,708.31,136703.83
5,190.0,39.3,20.6,7467.0,3914.0,809.58,153820.20


Check that the values for the new interaction features are correct for a few
of them.

In [62]:
# Write your code here.
data_inter = pd.DataFrame()
for i in range(data.shape[1]):
    for j in range(i+1, data.shape[1]):
        data_inter[f'{data.columns[i]} x {data.columns[j]}'] = data.iloc[:,i]*data.iloc[:,j]

avg_diff = abs(polynomial_regression[0].fit_transform(data).iloc[:,3:6].values - data_inter.values).max()
print(f'The maximum difference between the two dataframes is {avg_diff}')

data_inter.head()

The maximum difference between the two dataframes is 0.0


,Flipper Length (mm) x Culmen Length (mm),Flipper Length (mm) x Culmen Depth (mm),Culmen Length (mm) x Culmen Depth (mm)
0,7077.1,3384.7,731.17
1,7347.0,3236.4,687.30
2,7858.5,3510.0,725.40
4,7083.1,3724.9,708.31
5,7467.0,3914.0,809.58


Use the same cross-validation strategy as done previously to estimate the mean
and std of the MAE in grams (g) for such a pipeline. Compare with the results
without feature engineering.

In [28]:
# Write your code here.
scores = cross_val_score(polynomial_regression, data, target, cv=10, scoring="neg_mean_absolute_error")
scores = -scores
print(
    "The mean cross-validation MAE is: "
    f"{scores.mean():.3f} ± {scores.std():.3f} "
)

The mean cross-validation MAE is: 303.744 ± 48.773 



Now let's try to build an alternative pipeline with an adjustable number of
intermediate features while keeping a similar predictive power. To do so, try
using the `Nystroem` transformer instead of `PolynomialFeatures`. Set the
kernel parameter to `"poly"` and `degree` to 2. Adjust the number of
components to be as small as possible while keeping a good cross-validation
performance.

Hint: Use a `ValidationCurveDisplay` with `param_range = np.array([5, 10, 50,
100])` to find the optimal `n_components`.

In [31]:
# Write your code here.
from sklearn.kernel_approximation import Nystroem

nystroem_regression = make_pipeline(
    Nystroem(kernel="poly", degree=2, n_components=5, random_state=0),
    LinearRegression(),
)
nystroem_regression

Pipeline(steps=[('nystroem',
                 Nystroem(degree=2, kernel='poly', n_components=5,
                          random_state=0)),
                ('linearregression', LinearRegression())])

In [ ]:
# Write your code here.
import numpy as np
from sklearn.model_selection import ValidationCurveDisplay

n_components = np.array([5, 10, 50, 100])
disp = ValidationCurveDisplay.from_estimator(
    nystroem_regression,
    data,
    target,
    param_name="nystroem__n_components",
    param_range=n_components,
    cv=10,
    scoring="neg_mean_absolute_error",
    negate_score=True,
    n_jobs=2,
)
_ = disp.ax_.set(
    xlabel="# of components",
    ylabel="Mean absolute error (g)",
    title="Validation Curve for NyStroem",
)

How do the mean and std of the MAE for the Nystroem pipeline with optimal
`n_components` compare to the other previous models?

In [35]:
# Write your code here.
nystroem_regression.set_params(nystroem__n_components=10)

# Write your code here.
scores = cross_val_score(nystroem_regression, data, target, cv=10, scoring="neg_mean_absolute_error")
scores = -scores
print(
    "The mean cross-validation MAE is: "
    f"{scores.mean():.3f} ± {scores.std():.3f} "
)

The mean cross-validation MAE is: 299.066 ± 44.761 
